Ejercicio basado en https://www.kaggle.com/code/amolbhivarkar/knn-for-classification-using-scikit-learn/input?select=diabetes.csv

# CROSS VALIDATION

**En este ejemplo usaremos  el algoritmo k-Vecinos más cercanos para predecir si los pacientes en el "Conjunto de datos de diabetes de los indios Pima" tienen diabetes o no.

Comparamos el valor de K-vecinos usando una validación entrenamiento/test y luego aplicaremos Cross Validation para ver si ese valor cambia**

In [1]:
#Cargamos las librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
#Cargamos los datos
df = pd.read_csv('diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
#Veamos el tamaño del dataframe
df.shape

(768, 9)

Como se observó anteriormente, tenemos 768 filas y 9 columnas. Las primeras 8 columnas representan las características y la última columna representa el objetivo/etiqueta.

In [4]:
#Creamos la variable dependiente y la independiente
X = df.drop('Outcome',axis=1).values
y = df['Outcome'].values

Dividamos los datos aleatoriamente en conjuntos de entrenamiento y de prueba.

Ajustaremos/entrenaremos un clasificador en el conjunto de entrenamiento y haremos predicciones en el conjunto de prueba. Luego compararemos las predicciones con las etiquetas conocidas.



In [5]:
#importamos la función para separar
from sklearn.model_selection import train_test_split

Es una buena práctica realizar nuestra división de tal manera que refleje las etiquetas de los datos. En otras palabras, queremos que las etiquetas se dividan en el conjunto de entrenamiento y prueba tal como están en el conjunto de datos original. Entonces usamos el argumento de estratificar.

También creamos un conjunto de prueba de un tamaño de aproximadamente el 40% del conjunto de datos.

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=42, stratify=y)

Creemos un clasificador usando el algoritmo k-vecinos más cercanos.

Primero observemos primero las precisiones para diferentes valores de k.

In [7]:
#importamos el clasificador
from sklearn.neighbors import KNeighborsClassifier

#Configuramos matrices para almacenar la precisión de las pruebas y el entrenamiento
neighbors = np.arange(1,9)
train_accuracy =np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

for i,k in enumerate(neighbors):
    #
    knn = KNeighborsClassifier(n_neighbors=k)

    #
    knn.fit(X_train, y_train)

    #Calculamos la exactitud con entrenamiento
    train_accuracy[i] = knn.score(X_train, y_train)

    #Calculamos la exactitud con test
    test_accuracy[i] = knn.score(X_test, y_test)

In [ ]:
#Generamos un gráfico con las iteraciones con diferentes K vs la exactitud
plt.title('k-NN Varying number of neighbors')
plt.plot(neighbors, test_accuracy, label='Testing Accuracy')
plt.plot(neighbors, train_accuracy, label='Training accuracy')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.show()

Podemos observar arriba que obtenemos la máxima precisión de prueba para k = 7. Entonces, creemos un KNeighborsClassifier con un número de vecinos de 7.

In [9]:
#Seteamos el clasificador con 7 vecinos
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
#Ejecutamos
knn.fit(X_train,y_train)

In [ ]:
#Generamos el valor del SCORE del modelo
knn.score(X_test,y_test)

**Confusion Matrix**

Una matriz de confusión es una tabla que se utiliza a menudo para describir el rendimiento de un modelo de clasificación (o "clasificador") en un conjunto de datos de prueba cuyos valores verdaderos se conocen.

In [12]:
#
from sklearn.metrics import confusion_matrix

In [13]:
#realizamos predicciones
y_pred = knn.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred)

Considerando la matriz de confusión anterior:
Verdadero negativo = 165
Falso positivo = 36
Verdadero positivo = 60
Falso negativo = 47

La matriz de confusión también se puede obtener mediante el método de crosstab de pandas.

In [ ]:
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

**Classification Report**

Otro informe importante es el informe de Clasificación. Es un resumen de texto de la precisión, recuperación y puntuación F1 de cada clase.

In [16]:
#reporte de clasificación
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,y_pred))

**Cross Validation**

Ahora, antes de entrar en detalles sobre el ajuste de hiperparámetros, comprendamos el concepto de validación cruzada.

El rendimiento del modelo entrenado depende de la forma en que se dividen los datos. Puede que no sea representativo de la capacidad del modelo para generalizar.

La solución es la validación cruzada.

La validación cruzada es una técnica para evaluar modelos predictivos dividiendo la muestra original en un conjunto de entrenamiento para entrenar el modelo y un conjunto de prueba para evaluarlo.

En la validación cruzada de k veces, la muestra original se divide aleatoriamente en k submuestras de igual tamaño. De las k submuestras, una única submuestra se conserva como datos de validación para probar el modelo, y las k-1 submuestras restantes se utilizan como datos de entrenamiento. Luego, el proceso de validación cruzada se repite k veces (los pliegues), y cada una de las k submuestras se usa exactamente una vez como datos de validación. Los k resultados de los pliegues pueden luego promediarse (o combinarse de otro modo) para producir una estimación única. La ventaja de este método es que todas las observaciones se utilizan tanto para el entrenamiento como para la validación, y cada observación se utiliza para la validación exactamente una vez.

**Hyperparameter tuning**

El valor de k (es decir, 7) que seleccionamos anteriormente se seleccionó observando la curva de precisión frente al número de vecinos. Ésta es una forma primitiva de ajuste de hiperparámetros.

Hay una mejor manera de hacerlo que implica:

1) Probar varios valores de hiperparámetros diferentes

2) Montarlos todos por separado

3) Comprobar qué tan bien se desempeña cada uno

4) Elegir el que tenga mejor rendimiento

5) Usar validación cruzada siempre

Usaremos GridSearchCV de la librería Scikitlearn para busacr el mejor valor de K aplicando simultáneamente Cross Validation



In [23]:
#importando GridSearchCV
from sklearn.model_selection import GridSearchCV

In [24]:
#En el caso de un clasificador como knn, el parámetro a ajustar es n_neighbors
param_grid = {'n_neighbors':np.arange(1,50)}

In [ ]:
knn = KNeighborsClassifier()
knn_cv= GridSearchCV(knn,param_grid,cv=5)
knn_cv.fit(X,y)

In [ ]:
knn_cv.best_score_

El valor de exactitud anterior fue de 0.7305194805194806, menor al encontrado con GridSearchCV (0.7578558696205755) que usa Cross Validation. Veamos si cambia el número de K vecinos...

In [ ]:
knn_cv.best_params_

Por lo tanto, un clasificador knn con un número de vecinos de 14 logra la mejor puntuación/precisión de 0,7578, es decir, alrededor del 76 %.